# Generate data

In [1]:
import sys
sys.path.append("./script/")
from LogicAsker import *


## Generate 50 data for each tree node (208 total) on the skill tree

In [2]:

problems = ["inference", "contradiction", "unrelated"]
logics = ["propositional", "predicate"]
rule_categories = ["equivalent", "inference", "fallacy"]
cases = []
rule_dict = {"propositional equivalent": lib.PROPOSITIONAL_EQUIV_RULES,
             "propositional inference": lib.PROPOSITIONAL_INFERENCE_RULES,
             "propositional fallacy": lib.PROPOSITIONAL_FALLACY_RULES,
             "predicate equivalent": lib.QUANTIFIER_EQUIV_RULES,
             "predicate inference": lib.QUANTIFIER_INFERENCE_RULES,
             "predicate fallacy": lib.QUANTIFIER_FALLACY_RULES}

from tqdm.auto import tqdm
with tqdm(total=208) as pbar:
    for logic in logics:
        for rule_category in rule_categories:
            for problem in problems:
                rule_set = rule_dict[logic + " " + rule_category]
                rule_names = list(set([x.rule_name for x in rule_set]))
                for rule in rule_names:
                    rules = get_rules([rule])
                    if rule_category == "fallacy" and problem == "inference":
                        tmp = gen_cases(50, "3b", problem, 1, fallacies=rules)
                        cases += [[logic, rule_category, rule, problem, x[0], x[1]] for x in tmp]
                        # print(f"{logic} {rule_category} {rule} {problem}")
                        pbar.update(1)
                    elif rule_category != "fallacy" :
                        tmp = gen_cases(50, "3b", problem, 1, rules=rules)
                        cases += [[logic, rule_category, rule, problem, x[0], x[1]] for x in tmp]
                        # print(f"{logic} {rule_category} {rule} {problem}")
                        pbar.update(1)

# write cases into csv file
import csv
with open("./data_generated.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["logic", "rule_category", "rule", "problem", "query", "answer"])
    writer.writerows(cases)

  0%|          | 0/208 [00:00<?, ?it/s]

## Generate 50 data of different lengths

In [ ]:
import random
for length in tqdm(range(1, 11)):
    queries = []
    # propositional
    for i in range(41):
        cat = random.choice(["inference", "inference", "inference", "contradiction", "unrelated"])
        cases = gen_cases(1, "3b", cat, length, rules=lib.PROPOSITIONAL_EQUIV_RULES + lib.PROPOSITIONAL_INFERENCE_RULES)
        queries += [[x[0], x[1]] for x in cases]
    cases = gen_cases(9, "3b", "inference", length, rules=lib.PROPOSITIONAL_EQUIV_RULES + lib.PROPOSITIONAL_INFERENCE_RULES, fallacies=lib.PROPOSITIONAL_FALLACY_RULES)
    queries += [[x[0], x[1]] for x in cases]

    # predicate
    for i in range(41):
        cat = random.choice(["inference", "inference", "inference", "contradiction", "unrelated"])
        cases = gen_cases(1, "3b", cat, length, rules=lib.QUANTIFIER_EQUIV_RULES + lib.QUANTIFIER_INFERENCE_RULES)
        queries += [[x[0], x[1]] for x in cases]
    cases = gen_cases(9, "3b", "inference", length, rules=lib.QUANTIFIER_EQUIV_RULES + lib.QUANTIFIER_INFERENCE_RULES, fallacies=lib.QUANTIFIER_FALLACY_RULES)
    queries += [[x[0], x[1]] for x in cases]

    queries = pd.DataFrame(queries).rename(columns={0:"question", 1:"target"})
    queries.to_csv(f"./exp_len_{length}.csv", index=False)
